In [ ]:
import sys
sys.path.append("..")

In [ ]:
from emeval.input.spec_details import FileSpecDetails

sd = FileSpecDetails("data/", "shankari@eecs.berkeley.edu", "train_bus_ebike_mtv_ucb")

In [ ]:
from emeval.input.phone_view import PhoneView

pv = PhoneView(sd)

In [ ]:
import pandas as pd

def get_dfs(pv_map):
    """
    Returns a dictionary consisting of versions of the dataframes in a
    PhoneView map with uniquely-identifiable (i.e. fully queryable) rows.
    """
    df_map = {}
    pkey = set()
    
    def gen_pkey(node, curr_pkey):
        """
        Creates the primary key set, which consists of the columnns that
        will be pre-pended to each dataframe to uniquely identify each row.
        These columnns correspond to the terminal nodes of the PhoneView map.
        """
        for key, val in node.items():
            if isinstance(val, list):
                for e in val:
                    gen_pkey(e, curr_pkey)
            elif isinstance(val, dict):
                gen_pkey(val, curr_pkey)
            elif isinstance(val, pd.DataFrame):
                for col in val.columns.values:
                    if col in curr_pkey:
                        curr_pkey.remove(col)
                if key not in df_map:
                    df_map[key] = val
            else:
                curr_pkey.add(key)
    
    gen_pkey(pv_map, pkey)
    
    # TODO: figure out how to pre-pend rows to dataframes based on primary key
    
    # return df_map
    return pkey

get_dfs(pv.map())